动态语言和静态语言最大的不同，就是函数和类的定义，不是编译时定义的，而是运行时动态创建的。

静态语言在运行期创建类，必须构造源代码字符串再调用编译器，或者借助一些工具生成字节码实现

type()函数既可以返回一个对象的类型，又可以创建出新的类型。

In [1]:
class Hello(object):
    def hello(self, name='world'):
        print('Hello, %s.' % name)
        
h = Hello()
print(type(Hello))  #Hello是一个class，它的类型就是type
print(type(h))  #h是一个实例，它的类型就是class Hello

<class 'type'>
<class '__main__.Hello'>


class的定义是运行时动态创建的，而创建class的方法就是使用type()函数。

In [3]:
#通过type()函数创建出Hello类
def fn(self, name='world'): # 先定义函数
    print('Hello, %s.' % name)
    
Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

h = Hello()
h.hello()

Hello, world.


要创建一个class对象，type()函数依次传入3个参数：
1. class的名称；
2. 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；
3. class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。

通过type()函数创建的类和直接写class是完全一样的，因为Python解释器遇到class定义时，仅仅是扫描一下class定义的语法，然后调用type()函数创建出class。

metaclass（元类）：控制类的创建行为。可以把类看成是metaclass创建出来的“实例”。

先定义metaclass，就可以创建类，最后创建实例。

In [5]:
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

#给自定义的MyList增加一个add方法    
class MyList(list, metaclass=ListMetaclass):
    pass

l = MyList()
l.add('good')
print(l)

['good']


传入关键字参数metaclass，它指示Python解释器在创建MyList时，通过ListMetaclass.\__new\__()来创建，在此，我们可以修改类的定义，比如，加上新的方法，然后，返回修改后的定义。

\__new\__()方法接收到的参数依次是：

1. 当前准备创建的类的对象；
2. 类的名字；
3. 类继承的父类集合；
4. 类的方法集合。


ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。要编写一个ORM框架，所有的类都只能动态定义，因为只有使用者才能根据表的结构定义出对应的类来。

In [8]:
#1. 编写底层模块的第一步，就是先把调用接口写出来。
#父类Model和属性类型StringField、IntegerField是由ORM框架提供的
#剩下的方法比如save()全部由metaclass自动完成。虽然metaclass的编写会比较复杂，但ORM的使用者用起来却异常简单。
class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库：
u.save()



Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


In [9]:
#2. 按上面的接口来实现该ORM。
#Field负责保存数据库表的字段名和字段类型
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)
    
class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')  # python2
        #super().__init__(name, 'varchar(100)')  # python3
        
class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')  # python2
        #super().__init__(name, 'bigint')  # python3
        
        
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        print('cls: ',cls,'; name: ',name, '; bases : ',bases,'; attrs: ',attrs)
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)
    
class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

cls:  <class '__main__.ModelMetaclass'> ; name:  Model ; bases :  (<class 'dict'>,) ; attrs:  {'__module__': '__main__', '__qualname__': 'Model', '__init__': <function Model.__init__ at 0x10fb97730>, '__getattr__': <function Model.__getattr__ at 0x10fb976a8>, '__setattr__': <function Model.__setattr__ at 0x10fb97840>, 'save': <function Model.save at 0x10fb97950>, '__classcell__': <cell at 0x10fb06fd8: empty>}


当用户定义一个class User(Model)时，Python解释器首先在当前类User的定义中查找metaclass，如果没有找到，就继续在父类Model中查找metaclass，找到了，就使用Model中定义的metaclass的ModelMetaclass来创建User类，也就是说，metaclass可以隐式地继承到子类，但子类自己却感觉不到。

In [10]:
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)


class StringField(Field):
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100')


class IntegerField(Field):
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')


class Model(dict):
    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def save(self):
        fields = []
        params = []
        args = []
        for k in self.keys():
            fields.append(k)
            params.append('?')
            args.append(self[k])
        sql = 'insert into %s (%s) values (%s)' % (self.__class__.__name__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))


class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')


# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库
u.save()

SQL: insert into User (id,name,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']
